In [1]:

#GDELT API v2 documentation: https://blog.gdeltproject.org/gdelt-geo-2-0-api-debuts/


import pandas
import requests #use requests over 'urllib.request'. Work-around for encoding problem. See: https://stackoverflow.com/questions/47419104/unicodedecodeerror-utf-8-codec-cant-decode-python3
import json
from geojson import MultiPoint
import folium
from folium import IFrame #folium does not have a .element object/attribute see https://github.com/python-visualization/folium/issues/774
from bs4 import BeautifulSoup
import geopandas as gpd
from shapely.geometry import mapping, shape
from shapely.wkt import dumps, loads

from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame

#for reading from databases
import json
import psycopg2
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *

import getpass
import datetime

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
#GDELT GeoJson Endpoint
# url = "https://api.gdeltproject.org/api/v1/gkg_geojson?QUERY=Lang:Eng&MAXROWS=50&OUTPUTFIELDS=sumtotalmentions"
# url = "https://api.gdeltproject.org/api/v1/gkg_geojson?OUTPUTTYPE=2&OUTPUTFIELDS=sumtotalmentions&OUTPUTFIELDS=geores&OUTPUTFIELDS=themes"
url = "https://api.gdeltproject.org/api/v1/gkg_geojson?OUTPUTTYPE=2&OUTPUTFIELDS=sumtotalmentions,geores,themes,oneurl"
#Issue request to URL
request = requests.get(url)
print(request.status_code)
if request.status_code == 200: data = request.json()
print(type(data))
print("Rows loaded: "+str(len(data['features'])))

200
<class 'dict'>
Rows loaded: 464


In [3]:
#I think this solution works!
#However, this will probably just work for my data only.

flat_dictionary = {}
for feature in data['features']:
    for key, value in feature.items():
      if type(feature[key]) == dict:
        for k, v in feature[key].items():
            if k in flat_dictionary.keys():
                flat_dictionary[k].append(feature[key][k])
            else:
                flat_dictionary[k] = [v]
# print(flat_dictionary.keys())
                
df_features = pandas.DataFrame.from_dict(flat_dictionary)

#from: https://stackoverflow.com/questions/35491274/pandas-split-column-of-lists-into-multiple-columns
df_features[['lon','lat']] = pandas.DataFrame(df_features.coordinates.values.tolist(),index = df_features.index)
# df_features[['date','time']] = pandas.DataFrame(df_features.urlpubtimedate.values)
# df_features[urlpubtimedate] = df_features.apply(lambda x: x.split('T'))

print(list(df_features))
print(df_features.shape)

['allmentionedthemes', 'coordinates', 'geores', 'oneurl', 'sumtotalmentions', 'type', 'urlpubtimedate', 'lon', 'lat']
(464, 9)


In [4]:
# df.loc[df['column_name'] == some_value]
df_city_features = df_features.loc[df_features['geores'] == 3]


max_mentions = df_city_features.sumtotalmentions.max()
min_mentions = df_city_features.sumtotalmentions.min()
mean_lat = df_city_features.mean().lat
mean_lon = df_city_features.mean().lon

def getWideRadius(mentions):
    radRange = [4,50]
    rad = mentions/max_mentions*(radRange[1]-radRange[0])+radRange[0]
    return rad

def getRadius(mentions):
    radRange = [4,15]
    rad = mentions/max_mentions*(radRange[1]-radRange[0])+radRange[0]
    return rad
    

#map with folium


m = folium.Map(location=[mean_lat,mean_lon],zoom_start=2.5)
feature_group = folium.FeatureGroup(name='mentions')

for lat,lon,mentions,url in zip(df_city_features.lat,df_city_features.lon,df_city_features.sumtotalmentions,df_city_features.oneurl):
    folium.CircleMarker(
        radius=getWideRadius(mentions),
        location = [lat,lon],
        popup=folium.Popup("N Mentions: {}, {}".format(mentions,url)),
        color='crimson',
        fill=True,
        ).add_to(feature_group)
        
feature_group.add_to(m)
m

In [5]:
# implement GeoPandas
#tutorial found here: https://geohackweek.github.io/vector/04-geopandas-intro/

#make list of geometry features
geometry = [Point(xy) for xy in zip(df_city_features['lon'],df_city_features['lat'])]
# geometry = [Point(xy) for xy in zip(data['lon'],data['lat'])]

gdf = GeoDataFrame(df_city_features,geometry=geometry)
# data_list = [float(i) for i in list(gdf.coordinates.values)]
# print(data_list)
gdf.head(5)

,allmentionedthemes,coordinates,geores,oneurl,sumtotalmentions,type,urlpubtimedate,lon,lat,geometry
0,;SURVEILLANCE;MEDIA_MSM;TAX_FNCACT_DIRECTOR;TA...,"[-77.0003, 38.889]",3,http://www.newschannel10.com/story/37753252/re...,2,Point,2018-03-20T00:00:00Z,-77.0003,38.8890,POINT (-77.0003 38.889)
2,;TAX_FNCACT_LEADERS;GENERAL_GOVERNMENT;EPU_POL...,"[-99.0525, 19.6011]",3,https://laopinion.com/2018/03/19/politicos-mex...,2,Point,2018-03-20T00:00:00Z,-99.0525,19.6011,POINT (-99.05249999999999 19.6011)
4,;TAX_FNCACT_PRINCIPAL;;,"[113.131, 23.0268]",3,http://sc.stock.cnfol.com/ggzixun/20180320/261...,1,Point,2018-03-20T00:00:00Z,113.1310,23.0268,POINT (113.131 23.0268)
6,;SCIENCE;TAX_FNCACT_AUTHORITIES;EPU_POLICY_AUT...,"[8.55, 47.3667]",3,https://www.gutekueche.at/karotten-nudel-pfann...,2,Point,2018-03-20T00:00:00Z,8.5500,47.3667,POINT (8.550000000000001 47.3667)
8,,"[26.0167, 44.95]",3,https://www.cugetliber.ro/stiri-social-cartier...,2,Point,2018-03-20T00:00:00Z,26.0167,44.9500,POINT (26.0167 44.95)


In [6]:
# #add table to database and import data for MSU Denver
# import getpass #for hiding password information
# #CAUTION: whatever you input is still saved in memory and can be printed as a string to the console
# #perhaps shut everything down and/or overwrite/clear memory before pushing to git

# #reading data from a database!!
# #the tutorial reads a PostGIS database from Amazon cloud (awsome!) but I'm going to do it from MSU Denver DB

# #it's good practice to read database credentials from a separate json file
# #this way, code can be pushed to github without exposing sensitive information
# db_credentials_path = '/Users/Jacobus/Documents/Python/msu_denver_db_creds.json'
# with open(db_credentials_path,'r') as f:
#     db_conn_dict = json.load(f)
# db_conn_dict['password'] = getpass.getpass(prompt='Input Password (hidden): ')

In [7]:
#make list of geometry features (again because I delete it a few lines down)
geometry = [Point(xy) for xy in zip(df_city_features['lon'],df_city_features['lat'])]
gdf = GeoDataFrame(df_city_features,geometry=geometry)

#see for GeoAlchemy solution: https://gis.stackexchange.com/questions/239198/geopandas-dataframe-to-postgis-table-help
password = getpass.getpass(prompt='Password: ')
db_credentials_path = '/Users/Jacobus/Documents/Python/msu_denver_db_creds.json'
with open(db_credentials_path,'r') as f:
    db_conn_dict = json.load(f)
db_conn_dict['password'] = password

connection = psycopg2.connect(**db_conn_dict)

#read data from db
gdelt_mentions = gpd.read_postgis("select * from gdelt_mentions", connection, 
                            geom_col='geom', crs={'init':'epsg:4326'},
                            coerce_float=False)


connection.close()

#instantiate SQLAlchemy engine

engine = create_engine('postgresql://rjacobus:{}@gis4870.msudenver.edu:5432/rjacobus'.format(password))
#fix geometry column
gdf['geom'] = gdf['geometry'].apply(lambda x: WKTElement(x.wkt,srid=4326))
#drop the old geometry column
gdf.drop('geometry',1,inplace=True)

#append data to data table
gdf.to_sql('gdelt_mentions',engine, if_exists='append',index=False,
          dtype={
              'geom': Geometry('POINT',srid=4326),
              'urlpubtimedate': DateTime()
          })

#     if_exists = replace: If table exists, drop it, recreate it, and insert data.
#     if_exists = fail: If table exists, do nothing.
#     if_exists = append: If table exists, insert data. Create if does not exist.


gdelt_mentions

Password: ········


,allmentionedthemes,coordinates,geores,oneurl,sumtotalmentions,type,urlpubtimedate,lon,lat,geom
0,;TAX_FNCACT_WRITER;;,"{-1.3,43.8}",3,http://www.monacolife.net/monaco-issues-second...,1,Point,2018-02-27 17:45:00,-1.3000,43.8000,POINT (-1.3 43.8)
1,;TAX_ECON_PRICE;ECON_WORLDCURRENCIES_DOLLARS;E...,"{30.5167,50.4333}",3,http://podrobnosti.ua/2228507-sutichki-blja-ve...,17,Point,2018-02-27 17:45:00,30.5167,50.4333,POINT (30.5167 50.4333)
2,;TAX_FNCACT_GUIDE;EDUCATION;TAX_FNCACT_TEACHER...,"{98.9847,18.7904}",3,http://praew.com/people/148265.html,1,Point,2018-02-27 17:45:00,98.9847,18.7904,POINT (98.9847 18.7904)
3,;NATURAL_DISASTER_HEAVY_RAIN;CRISISLEX_O01_WEA...,"{9.9447,3.7553}",3,http://www.cameroon-one.com/site/news/index.ph...,1,Point,2018-02-27 17:45:00,9.9447,3.7553,POINT (9.944699999999999 3.7553)
4,;AFFECT;TAX_ETHNICITY_GERMAN;TAX_WORLDLANGUAGE...,"{11.5833,48.15}",3,http://www.correiodopovo.com.br/Noticias/Inter...,1,Point,2018-02-27 17:45:00,11.5833,48.1500,POINT (11.5833 48.15)
5,;TAX_FNCACT_MANAGER;TAX_FNCACT_CORRESPONDENT;C...,"{17.0232,47.4912}",3,https://www.hirado.hu/belfold/kozelet/cikk/201...,2,Point,2018-02-27 17:45:00,17.0232,47.4912,POINT (17.0232 47.4912)
6,;WB_1104_MACROECONOMIC_VULNERABILITY_AND_DEBT;...,"{-0.1167,51.5}",3,https://www.newcriterion.com/blogs/dispatch/th...,14,Point,2018-02-27 17:45:00,-0.1167,51.5000,POINT (-0.1167 51.5)
7,;TAX_FNCACT_EMPLOYERS;WB_2690_CATEGORIES_OF_EM...,"{7.1,50.7333}",3,https://www.welt.de/regionales/nrw/article1740...,1,Point,2018-02-27 17:45:00,7.1000,50.7333,POINT (7.1 50.7333)
8,;TAX_FNCACT_MANAGER;TAX_FNCACT_CORRESPONDENT;T...,"{16.6552,47.7141}",3,https://www.hirado.hu/belfold/kozelet/cikk/201...,2,Point,2018-02-27 17:45:00,16.6552,47.7141,POINT (16.6552 47.7141)
9,;TAX_FNCACT_DEPUTY;MANMADE_DISASTER_IMPLIED;NA...,"{-49.25,-25.4167}",3,http://hojeemdia.com.br/primeiro-plano/pol%C3%...,1,Point,2018-02-27 17:45:00,-49.2500,-25.4167,POINT (-49.25 -25.4167)


In [8]:
#run here to map data table from msudenver PostGIS

#see for GeoAlchemy solution: https://gis.stackexchange.com/questions/239198/geopandas-dataframe-to-postgis-table-help
password = getpass.getpass(prompt='Password: ')
db_credentials_path = '/Users/Jacobus/Documents/Python/msu_denver_db_creds.json'
with open(db_credentials_path,'r') as f:
    db_conn_dict = json.load(f)
db_conn_dict['password'] = password

connection = psycopg2.connect(**db_conn_dict)

#read data from db
gdelt_mentions = gpd.read_postgis("select * from gdelt_mentions WHERE urlpubtimedate > '2018-03-05'", connection, 
                            geom_col='geom', crs={'init':'epsg:4326'},
                            coerce_float=False)


connection.close()

df = gdelt_mentions

max_mentions = df.sumtotalmentions.max()
min_mentions = df.sumtotalmentions.min()
mean_lat = df.mean().lat
mean_lon = df.mean().lon

def getWideRadius(mentions):
    radRange = [4,50]
    rad = mentions/max_mentions*(radRange[1]-radRange[0])+radRange[0]
    return rad

def getRadius(mentions):
    radRange = [4,15]
    rad = mentions/max_mentions*(radRange[1]-radRange[0])+radRange[0]
    return rad
    

#map with folium


m = folium.Map(location=[mean_lat,mean_lon],zoom_start=2.5)
feature_group = folium.FeatureGroup(name='mentions')

for lat,lon,mentions,url in zip(df.lat,df.lon,df.sumtotalmentions,df.oneurl):
    folium.CircleMarker(
        radius=getWideRadius(mentions),
        location = [lat,lon],
        popup=folium.Popup("N Mentions: {}, {}".format(mentions,url)),
        color='crimson',
        fill=True,
        ).add_to(feature_group)
        
feature_group.add_to(m)
m

Password: ········


In [10]:
#database info

password = getpass.getpass(prompt='Password: ')
db_credentials_path = '/Users/Jacobus/Documents/Python/msu_denver_db_creds.json'
with open(db_credentials_path,'r') as f:
    db_conn_dict = json.load(f)
db_conn_dict['password'] = password

connection = psycopg2.connect(**db_conn_dict)

#read data from db
gdelt_mentions = gpd.read_postgis("select * from gdelt_mentions", connection, 
                            geom_col='geom', crs={'init':'epsg:4326'},
                            coerce_float=False)


connection.close()

gdelt_mentions.info()

Password: ········
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1947 entries, 0 to 1946
Data columns (total 10 columns):
allmentionedthemes    1947 non-null object
coordinates           1947 non-null object
geores                1947 non-null int64
oneurl                1947 non-null object
sumtotalmentions      1947 non-null int64
type                  1947 non-null object
urlpubtimedate        1947 non-null datetime64[ns]
lon                   1947 non-null float64
lat                   1947 non-null float64
geom                  1947 non-null object
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 152.2+ KB


In [ ]:
# #The above method using sqlalchemy may be better for quickly creating/updating tables

# #connect to database returning a connection object
# connection = psycopg2.connect(**db_conn_dict)

# #first creat a cursor object on the current connection
# cursor = connection.cursor()

# #create table
# create_table_command = (
# """
# CREATE TABLE gdelt_mentions (
#     id INTEGER PRIMARY KEY,
#     all_themes TEXT,
#     one_url TEXT,
#     sum_mentions INTEGER,
#     urlpubtimedate TIMESTAMP,
#     geometry FLOAT
# )
# """)

# update_data_command = (
# """
# UPDATE gdelt_mentions
#     SET all_themes = %s,
#         one_url = %s,
#         sum_mentions = %s,
#         urlpubtimedate = %s,
#         geometry = %s    
# """
# )

# #what datatype to I use to insert values into the SQL?
# cursor.execute(update_data_command,(list(gdf.allmentionedthemes.values), 
#                                     list(gdf.oneurl.values), 
#                                     list(gdf.sumtotalmentions.values), 
#                                     list(gdf.urlpubtimedate.values), 
#                                     list(gdf.geometry.values)))

# #close the cursor
# cursor.close()
# #commit the changes
# connection.commit()


# #close the connection once all desired data has been extracted
# connection.close()
